In [1]:
# Setup and Installation
from datetime import datetime
import yfinance as yf
import time
import pandas as pd

In [2]:
class BaseTradingStrategy:
    def __init__(self, name):
        self.__name = name

    # This method is to be overridden by Child Classes
    def generate_signal(self, price_data):
        return "Hold"

    @property
    def name(self):
        return self.__name

    @name.setter
    def name(self, new_name):
        self.__name = new_name


In [3]:
# Simple Moving Average
class SMATradingStrategy(BaseTradingStrategy):
    def __init__(self, name, sWindow, lWindow):
        super().__init__(name or "SMA Trading Strategy")
        self.__sWindow = sWindow
        self.__lWindow = lWindow

    def generate_signal(self, price_data):
        if len(price_data[-self.__lWindow:]) < self.__lWindow:
            return "Hold"
        short_avg = sum(price_data[-self.__sWindow:])/self.__sWindow
        long_avg = sum(price_data[-self.__lWindow:])/self.__lWindow
        if short_avg > long_avg:
            return "Buy"
        elif short_avg < long_avg:
            return "Sell"
        else:
            return "Hold"

    @property
    def sWindow(self):
        return self.__sWindow

    @sWindow.setter
    def sWindow(self, new_sWindow):
        if isinstance(new_sWindow, (int, float)):
            if new_sWindow < self.__lWindow:
                self.__sWindow = new_sWindow
            else:
                print(f"Input must be less than long Window, which is currently {self.__lWindow}")
        else:
            print("Please provide the new threshold as a number")

    @property
    def lWindow(self):
        return self.__lWindow

    @lWindow.setter
    def lWindow(self, new_lWindow):
        if isinstance(new_lWindow, (int, float)):
            if new_lWindow > self.__sWindow:
                self.__lWindow = new_lWindow
            else:
                print(f"Input must be greater than short Window, which is currently {self.__sWindow}")
        else:
            print("Please provide the new threshold as a number")


sma = SMATradingStrategy("SMA", 2, 5)
name = sma.name
signal = sma.generate_signal([1, 2, 3, 4, 5, 19, 7, 8, 9, 10])

In [4]:
# Trade Manager

class TradeManager:
    def __init__(self, strategy_name, signal, quantity):
        self.__strategy_name = strategy_name
        self.__signal = signal
        self.__quantity = quantity
        self.__timestamp = datetime.now()

    def execute(self):
        print(f"Executed {self.__signal} trade with strategy: {self.__strategy_name}, with quantity: {self.__quantity} at {self.__timestamp}.")

    @property
    def strategy_name(self):
        return self.__strategy_name

    @property
    def signal(self):
        return self.__signal

    @property
    def quantity(self):
        return self.__quantity

    @property
    def timestamp(self):
        return self.__timestamp

my_trade = TradeManager(name, signal, 34)
print(my_trade.strategy_name)
print(my_trade.signal)
print(my_trade.quantity)
print(my_trade.timestamp)
my_trade.execute()

SMA
Sell
34
2025-06-20 22:20:40.388741
Executed Sell trade with strategy: SMA, with quantity: 34 at 2025-06-20 22:20:40.388741.


In [5]:
# Fetching live data from the exchange

class MockTradingApi:
    def __init__(self, balance):
        self.__balance = balance

    def place_order(self, trade, price):
        action = trade.signal
        total_price = trade.quantity * price
        if action == "Buy":
            if self.__balance >= total_price:
                self.__balance -= total_price
                print(f"Placed a Buy trade at {price}. Remaining Balance: ${self.__balance}")
            else:
                print("You do not have sufficient balance to execute this trade")
        elif action == "Sell":
            self.__balance += total_price
            print(f"Placed a Sell trade at {price}. Remaining Balance: ${self.__balance}")
        else:
            print("No trading at the moment")

    @property
    def balance(self):
        return self.__balance

mock_api = MockTradingApi(1000)
mock_api.place_order(my_trade, 40)

Placed a Sell trade at 40. Remaining Balance: $2360


In [6]:
# Integrator

class TradingSystem:
    def __init__(self, api, strategy, symbol):
        self.__api = api
        self.__strategy = strategy
        self.__symbol = symbol
        self.__price_data = []

    def fetch_price_data(self):
        data = yf.download(tickers=self.__symbol, period="1d", interval="1m", auto_adjust=True)
        if not data.empty:
            # If Close is a DataFrame (e.g., multiple tickers), pick the correct column
            if isinstance(data["Close"], pd.DataFrame):
                price = data["Close"][self.__symbol].iloc[-1]
            else:
                price = data["Close"].iloc[-1]
            self.__price_data.append(price)
            if len(self.__price_data) > self.__strategy.lWindow:
                self.__price_data.pop(0)
            print(f"Fetched new price data: ${price}")
            print(self.__price_data)
        else:
            print("No data fetched")

    def run(self):
        self.fetch_price_data()
        signal = self.__strategy.generate_signal(self.__price_data)
        print(f"Generated Signal: {signal}")
        if signal in ["Buy", "Sell"]:
            trade = TradeManager(self.__strategy.name, signal, 1) 
            trade.execute()
            self.__api.place_order(trade, self.__price_data[-1])
            
    @property
    def api(self):
        return self.__api

    @property
    def strategy(self):
        return self.__strategy

    @property
    def symbol(self):
        return self.__symbol

    @property
    def price_data(self):
        return self.__price_data

In [7]:
api = MockTradingApi(balance=12080.0)
sma = SMATradingStrategy("SMA", 2, 5)
symbol = "AAPL"
trading_system = TradingSystem(api, sma, symbol)
for _ in range(5):
    trading_system.run()
    print(f"Remaining balance: {api.balance}")
    time.sleep(2)

[*********************100%***********************]  1 of 1 completed


Fetched new price data: $200.94000244140625
[200.94000244140625]
Generated Signal: Hold
Remaining balance: 12080.0


[*********************100%***********************]  1 of 1 completed


Fetched new price data: $200.94000244140625
[200.94000244140625, 200.94000244140625]
Generated Signal: Hold
Remaining balance: 12080.0


[*********************100%***********************]  1 of 1 completed


Fetched new price data: $200.94000244140625
[200.94000244140625, 200.94000244140625, 200.94000244140625]
Generated Signal: Hold
Remaining balance: 12080.0


[*********************100%***********************]  1 of 1 completed


Fetched new price data: $200.94000244140625
[200.94000244140625, 200.94000244140625, 200.94000244140625, 200.94000244140625]
Generated Signal: Hold
Remaining balance: 12080.0


[*********************100%***********************]  1 of 1 completed


Fetched new price data: $200.94000244140625
[200.94000244140625, 200.94000244140625, 200.94000244140625, 200.94000244140625, 200.94000244140625]
Generated Signal: Hold
Remaining balance: 12080.0


In [8]:
data = yf.download(tickers="AAPL", period="1d", interval="1m")
price = data["Close"].iloc[-1]
print(price)

C:\Users\emmanuel.osademe\AppData\Local\Temp\ipykernel_232\1896774579.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers="AAPL", period="1d", interval="1m")
[*********************100%***********************]  1 of 1 completed

Ticker
AAPL    200.940002
Name: 2025-06-20 19:59:00+00:00, dtype: float64
